In [ ]:
from raster_to_tile import data_split

tif_selva = r"D:\Shapefiles\PI\CO_2512022106159\VOL_PER1_ORT_001_010438\IMG_PER1_ORT_PMS_010438\IMG_PER1_20230922151914_ORT_PMS_010438.TIF"
tif_sierra = r"D:\Shapefiles\PI\CO_2510221339013\VOL_PER1_ORT_001_002705\IMG_PER1_ORT_PMS_002705\IMG_PER1_20210706153514_ORT_PMS_002705.TIF"

mask_casas_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\buildings_32718_selva.gpkg"
mask_casas_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\buildings_32717_sierra.gpkg"

mask_caminos_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\capa_carreteras_selva.gpkg"
mask_caminos_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\\capa_carreteras_sierra.gpkg"

gpkg_division_selva_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSelva\division_en_tile_selva.gpkg"
gpkg_division_sierra_tile = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\division_en_tile_sierra.gpkg"

dataset_selva_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSelvaMulticlase"
dataset_sierra_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase"

### Area Excluida
gpkg_area_ciudad = r"D:\Diciembre\Entrenamiento\training_road_model\CapasSierra\area_excluida_sierra.gpkg"

data_split(tif_sierra, mask_casas_sierra_path, mask_caminos_sierra_path, gpkg_division_sierra_tile, dataset_sierra_path, gpkg_area_excluida_path=gpkg_area_ciudad)

In [ ]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices('GPU'))


In [16]:
from utils2 import split_dataset
dataset_path = r"D:\Diciembre\Entrenamiento\training_road_model\DatasetSierraMulticlase"

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset(carpeta=dataset_path)


División correcta sin errores de archivos.
--------------------------------------------------
TRAIN     → imágenes: 4117 | máscaras: 4117
VALIDATION → imágenes: 938 | máscaras: 938
TEST      → imágenes: 969 | máscaras: 969
--------------------------------------------------
TOTAL IMGS: 6024
TOTAL MASKS: 6024
--------------------------------------------------


In [ ]:
from TiffDataGeneratorAugmentedUnet import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, normalize='imagenet',batch_size=1, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


In [18]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [19]:
def masked_sparse_cce(y_true, y_pred):
    """
    Sparse categorical crossentropy que ignora píxeles con label 255
    """
    y_true = tf.squeeze(y_true, axis=-1)  # (B,H,W)
    y_true = tf.cast(y_true, tf.int32)
    # Crear máscara para píxeles válidos
    mask = tf.not_equal(y_true, 255)
    mask = tf.cast(mask, tf.float32)
    
    # Reemplazar 255 con 0 para evitar errores
    y_true_safe = tf.where(mask > 0, y_true, tf.zeros_like(y_true))
    
    # Calcular loss por píxel
    loss = tf.keras.losses.sparse_categorical_crossentropy(
        y_true_safe, y_pred, from_logits=False
    )
    
    # Aplicar máscara y promediar solo sobre píxeles válidos
    loss = loss * mask
    
    # Evitar división por cero
    num_valid = tf.maximum(tf.reduce_sum(mask), 1.0)
    
    return tf.reduce_sum(loss) / num_valid

In [20]:
class MaskedIoUPerClass(tf.keras.metrics.Metric):
    def __init__(self, num_classes=3, class_names=None, name='masked_iou_per_class', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.class_names = class_names or [f'class_{i}' for i in range(num_classes)]
        
        # Matriz de confusión acumulativa
        self.total_cm = self.add_weight(
            name='total_confusion_matrix',
            shape=(num_classes, num_classes),
            initializer='zeros',
            dtype=tf.float32
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convertir a int32
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)
        
        # Máscara para ignorar 255
        mask = tf.not_equal(y_true, 255)
        
        # Aplicar máscara
        y_true = tf.boolean_mask(y_true, mask)
        y_pred = tf.boolean_mask(y_pred, mask)
        
        # Calcular confusion matrix
        current_cm = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )
        
        self.total_cm.assign_add(current_cm)
    
    def result(self):
        """Retorna el mIoU promedio (para compatibilidad con callbacks)"""
        iou_per_class = self._compute_iou_per_class()
        return tf.reduce_mean(iou_per_class)
    
    def _compute_iou_per_class(self):
        """Calcula IoU para cada clase"""
        sum_over_row = tf.reduce_sum(self.total_cm, axis=0)  # Predicciones
        sum_over_col = tf.reduce_sum(self.total_cm, axis=1)  # Ground truth
        diag = tf.linalg.diag_part(self.total_cm)  # True positives
        
        # IoU = TP / (TP + FP + FN)
        denominator = sum_over_row + sum_over_col - diag
        
        iou = tf.where(
            denominator > 0,
            diag / denominator,
            0.0
        )
        
        return iou
    
    def get_iou_per_class(self):
        """Método para obtener IoU por clase (usar después del entrenamiento)"""
        iou = self._compute_iou_per_class()
        return {self.class_names[i]: float(iou[i].numpy()) for i in range(self.num_classes)}
    
    def reset_state(self):
        self.total_cm.assign(tf.zeros_like(self.total_cm))

In [21]:
class MaskedSparseCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, num_classes=3, class_names=None, name='masked_acc', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.class_names = class_names or [f'class_{i}' for i in range(num_classes)]
        
        # Acumuladores globales
        self.total_correct = self.add_weight(name='total_correct', initializer='zeros', dtype=tf.float32)
        self.total_pixels = self.add_weight(name='total_pixels', initializer='zeros', dtype=tf.float32)
        
        # Acumuladores por clase
        self.correct_per_class = self.add_weight(
            name='correct_per_class',
            shape=(num_classes,),
            initializer='zeros',
            dtype=tf.float32
        )
        self.pixels_per_class = self.add_weight(
            name='pixels_per_class',
            shape=(num_classes,),
            initializer='zeros',
            dtype=tf.float32
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convertir a int32
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)
        
        # Máscara para ignorar 255
        mask = tf.not_equal(y_true, 255)
        
        # Aplicar máscara
        y_true_masked = tf.boolean_mask(y_true, mask)
        y_pred_masked = tf.boolean_mask(y_pred, mask)
        
        # Accuracy global
        correct = tf.cast(tf.equal(y_true_masked, y_pred_masked), tf.float32)
        self.total_correct.assign_add(tf.reduce_sum(correct))
        self.total_pixels.assign_add(tf.cast(tf.size(y_true_masked), tf.float32))
        
        # Accuracy por clase
        for class_id in range(self.num_classes):
            # Máscara para la clase actual
            class_mask = tf.equal(y_true_masked, class_id)
            
            if tf.reduce_any(class_mask):
                y_true_class = tf.boolean_mask(y_true_masked, class_mask)
                y_pred_class = tf.boolean_mask(y_pred_masked, class_mask)
                
                correct_class = tf.cast(tf.equal(y_true_class, y_pred_class), tf.float32)
                
                # Actualizar acumuladores
                self.correct_per_class.scatter_add(
                    tf.IndexedSlices(tf.reduce_sum(correct_class), [class_id])
                )
                self.pixels_per_class.scatter_add(
                    tf.IndexedSlices(tf.cast(tf.size(y_true_class), tf.float32), [class_id])
                )
    
    def result(self):
        """Retorna accuracy global (para compatibilidad con callbacks)"""
        return self.total_correct / tf.maximum(self.total_pixels, 1.0)
    
    def get_accuracy_per_class(self):
        """Obtiene accuracy por clase"""
        acc_per_class = tf.where(
            self.pixels_per_class > 0,
            self.correct_per_class / self.pixels_per_class,
            0.0
        )
        return {self.class_names[i]: float(acc_per_class[i].numpy()) for i in range(self.num_classes)}
    
    def reset_state(self):
        self.total_correct.assign(0.0)
        self.total_pixels.assign(0.0)
        self.correct_per_class.assign(tf.zeros((self.num_classes,)))
        self.pixels_per_class.assign(tf.zeros((self.num_classes,)))

In [22]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=3,
    activation='softmax'
    #input_shape=(512, 512, 4)
)

# Definir nombres de clases
class_names = ['background', 'road', 'building']

model.compile(
    optimizer=Adam(1e-4),
    loss=masked_sparse_cce,
    metrics=[
        MaskedSparseCategoricalAccuracy(num_classes=3, class_names=class_names, name='acc'),
        MaskedIoUPerClass(num_classes=3, class_names=class_names, name='miou')
    ]
)


In [23]:
class PrintMetricsPerClass(tf.keras.callbacks.Callback):
    def __init__(self, metric_names=['acc', 'miou']):
        super().__init__()
        self.metric_names = metric_names
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n📊 Época {epoch+1} - Métricas por clase:")
        
        for metric in self.model.metrics:
            if metric.name in self.metric_names:
                if hasattr(metric, 'get_accuracy_per_class'):
                    results = metric.get_accuracy_per_class()
                    print(f"\n  Accuracy por clase:")
                    for class_name, value in results.items():
                        print(f"    {class_name}: {value:.4f}")
                
                elif hasattr(metric, 'get_iou_per_class'):
                    results = metric.get_iou_per_class()
                    print(f"\n  IoU por clase:")
                    for class_name, value in results.items():
                        print(f"    {class_name}: {value:.4f}")

In [24]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Early stopping: detener cuando el modelo deje de mejorar
early_stop = EarlyStopping(
    monitor='val_loss',      # métrica que se evalúa
    mode='min',           # porque cuanto más alto, mejor
    patience=10,
    restore_best_weights=True,
    verbose= 1
)

# Guardar el mejor modelo automáticamente
checkpoint = ModelCheckpoint(
    r"checkpoint.keras",         # ruta del archivo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop, checkpoint, reduce_lr, PrintMetricsPerClass()]
)
print("Epochs realizadas:", len(history.history['loss']))

In [ ]:
# Después del entrenamiento, obtener IoU por clase
iou_metric = model.metrics[-1]  # Última métrica
iou_per_class = iou_metric.get_iou_per_class()

print("\n📊 IoU por clase:")
for class_name, iou_value in iou_per_class.items():
    print(f"  {class_name}: {iou_value:.4f}")

In [ ]:
# Save final model
model.save("model_final.keras")

# Evaluate on train set
print("\n=== Evaluación en test set ===")
train_results = model.evaluate(train_gen, verbose=1)
print(f"Train Loss: {train_results[0]:.4f}")
print(f"Train Masked Accuracy: {train_results[1]:.4f}")
print(f"Train Masked Mean IoU: {train_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en val set ===")
val_results = model.evaluate(val_gen, verbose=1)
print(f"Val Loss: {val_results[0]:.4f}")
print(f"Val Masked Accuracy: {val_results[1]:.4f}")
print(f"Val Masked Mean IoU: {val_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en test set ===")
test_results = model.evaluate(test_gen, verbose=1)
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Masked Accuracy: {test_results[1]:.4f}")
print(f"Test Masked Mean IoU: {test_results[2]:.4f}")